<!--
authors: Matthew Wilson, Daniele Reda
created: 2020/01/14
last_updated: 2023/02/08
-->


## CPSC 533V: Assignment 2 - Tabular Q Learning and DQN (Due Thu Feb 22)

- Name: **Yuwei Yin**
- Student ID: **36211928**
- Email: **yuweiyin@cs.ubc.ca**
- Institute: Department of Computer Science, University of British Columbia

GitHub: <a>https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/</a>

---

#  Part 1 [54 pts] Tabular Q-Learning 

Tabular Q-learning is an RL algorithm for problems with discrete states and discrete actions. The algorithm is described in the class notes, which borrows the summary description from [Section 6.5](http://incompleteideas.net/book/RLbook2018.pdf#page=153) of Richard Sutton's RL book. In the tabular approach, the Q-value is represented as a lookup table. As discussed in class, Q-learning can further be extended to continuous states and discrete actions, leading to the [Atari DQN](https://arxiv.org/abs/1312.5602) / Deep Q-learning algorithm.  However, it is important and informative to first fully understand tabular Q-learning.

Informally, Q-learning works as follows: The goal is to learn the optimal Q-function: 
`Q(s,a)`, which is the *value* of being at state `s` and taking action `a`.  Q tells you how well you expect to do, on average, from here on out, given that you act optimally.  Once the Q function is learned, choosing an optimal action is as simple as looping over all possible actions and choosing the one with the highest Q (optimal action $a^* = \text{argmax}_a Q(s,a)$).  To learn Q, we initialize it arbitrarily and then iteratively refine it using the Bellman backup equation for Q functions, namely: 
$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \text{max}_a Q(s', a) - Q(s,a)]$.
Here, $r$ is the reward associated with with the transition from state s to s', and $\alpha$ is a learning rate.

In the first part of assignment you will implement tabular Q-learning and apply it to CartPole -- an environment with a **continuous** state space.  To apply the tabular method, you will need to discretize the CartPole state space by dividing the state-space into bins.


**Goals:**
- to become familiar with python/numpy, as well as using an OpenAI Gym environment
- to understand tabular Q-learning, by implementing tabular Q-Learning for 
  a discretized version of a continuous-state environment, and experimenting with the implementation
- (optional) to develop further intuition regarding possible variations of the algorithm

## Introduction
Deep reinforcement learning has generated impressive results for board games ([Go][go], [Chess/Shogi][chess]), video games ([Atari][atari], [DOTA2][dota], [StarCraft II][scii]), [and][baoding] [robotic][rubix] [control][anymal] ([of][cassie] [course][mimic] ;)).  RL is beginning to work for an increasing range of tasks and capabilities.  At the same time, there are many [gaping holes][irpan] and [difficulties][amid] in applying these methods. Understanding deep RL is important if you wish to have a good grasp of the modern landscape of control methods.

These next several assignments are designed to get you started with deep reinforcement learning, to give you a more close and personal understanding of the methods, and to provide you with a good starting point from which you can branch out into topics of interest. You will implement basic versions of some of the important fundamental algorithms in this space, including Q-learning and policy gradient/search methods.

We will only have time to cover a subset of methods and ideas in this space.
If you want to dig deeper, we suggest following the links given on the course webpage.  Additionally we draw special attention to the [Sutton book](http://incompleteideas.net/book/RLbook2018.pdf) for RL fundamentals and in depth coverage, and OpenAI's [Spinning Up resources](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html) for a concise intro to RL and deep RL concepts, as well as good comparisons and implementations of modern deep RL algorithms.


[atari]: https://arxiv.org/abs/1312.5602
[go]: https://deepmind.com/research/case-studies/alphago-the-story-so-far
[chess]:https://deepmind.com/blog/article/alphazero-shedding-new-light-grand-games-chess-shogi-and-go 
[dota]: https://openai.com/blog/openai-five/
[scii]: https://deepmind.com/blog/article/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning
[baoding]: https://bair.berkeley.edu/blog/2019/09/30/deep-dynamics/
[rubix]: https://openai.com/blog/solving-rubiks-cube/
[cassie]: https://www.cs.ubc.ca/~van/papers/2019-CORL-cassie/index.html
[mimic]: https://www.cs.ubc.ca/~van/papers/2018-TOG-deepMimic/index.html
[anymal]: https://arxiv.org/abs/1901.08652


[irpan]: https://www.alexirpan.com/2018/02/14/rl-hard.html
[amid]: http://amid.fish/reproducing-deep-rl



In [ ]:
# Install Python3 packages
# !pip install notebook
!pip install numpy
!pip install gym
!pip install gymnasium
!pip install torch
!pip install pygame
!pip install ipdb

# # # uncomment if necesary
# !pip install numpy
# # !pip install gym
# # # OR:
# !pip install gymnasium

In [1]:
# Import Python3 packages
import os
import time
import random
import itertools

import numpy as np
# import gym
import gymnasium as gym
import torch

# Ramdom Seed
RANDOM_SEED = 42

def set_seed(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(RANDOM_SEED)

---

## [12 pts] Explore the CartPole environment 

Your first task is to familiarize yourself with the OpenAI gym interface and the [CartPole environment](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py)
by writing a simple hand-coded policy to try to solve it.  
To begin understanding OpenAI Gym environments, [read this first](https://gymnasium.farama.org/api/env/).) 
The gym interface is very popular and you will see many algorithm implementations and 
custom environments that support it.  You may even want to use the API in your course projects, 
to define a custom environment for a task you want to solve.

Note that there were several breaking changes introduced in the past few years to the gym API. Some reference algorithm implementations online might be using the old version:
- `obs = env.reset()` $\to$  `obs, info = env.reset()`
- `obs, reward, done, info = env.step(action)` $\to$ `obs, reward, terminated, truncated, info = env.step(action)`
- `env.render()` no longer accepts the `render_mode` parameter (e.g. human mode where the environment is rendered in a pop-up window, or rgb_array which allows headless conversion to images or videos)

Below is some example code that runs a simple random policy.  You are to:
- **run the code to see what it does**
- **write code that chooses an action based on the observation**.  You will need to learn about the gym API and to read the CartPole documentation to figure out what the `action` and `obs` vectors mean for this environment. 
Your hand-coded policy can be arbitrary, and it should ideally do better than the random policy.  There is no single correct answer. The goal is to become familiar with `env`s.
- **write code to print out the total reward gained by your policy in a single episode run**
- **answer the short-response questions below** (see the TODOs for all of this)

In [2]:
env = gym.make("CartPole-v1", render_mode="human")  # you can also try LunarLander-v2, but make sure to change it back
print(f"observation space:\n{env.observation_space}")
print(f"action space:\n{env.action_space}")

# To find out what the observations mean, read the CartPole documentation.
# Uncomment the lines below, or visit the source file: 
# https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py

#cartpole = env.unwrapped
#cartpole?

observation space:
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space:
Discrete(2)


In [3]:
# 1.1 [10pts] (Part 1/3)

# runs a single episode and render it.  try running this before editing anything
obs, info = env.reset(seed=RANDOM_SEED)  # get first obs/state; set random seed for the env
rewards_sum = 0  # The sum of rewards

while True:
    # Replace this `action` with something that depends on `obs` 
    action = env.action_space.sample()  # random action

    obs, reward, terminated, truncated, info = env.step(action)
    rewards_sum += reward  # reward cumulation

    env.render()
    time.sleep(0.1)  # so it doesn't render too quickly

    if terminated | truncated:
        break

env.close()

# Print out your total sum of rewards here
print(f"[Random Policy] The total reward: {rewards_sum}")

[Random Policy] The total reward: 24.0


In [4]:
# 1.1 [10pts] (Part 2/3)

max_episode = 10  # run several times
# max_rewards = 0
rewards_list = []

for ep_idx in range(max_episode):
    env = gym.make("CartPole-v1", render_mode="human")
    # runs a single episode and render it. Try running this before editing anything
    obs, info = env.reset(seed=RANDOM_SEED + ep_idx)  # get first obs/state; set random seed for the env
    rewards_sum = 0  # The sum of rewards in this episode
    while True:
        # Replace this `action` with something that depends on `obs` 
        action = env.action_space.sample()  # random action

        obs, reward, terminated, truncated, info = env.step(action)
        rewards_sum += reward  # reward cumulation

        env.render()
        time.sleep(0.1)  # so it doesn't render too quickly

        if terminated | truncated:
            break

    # Done simulation of this episode
    rewards_list.append(rewards_sum)
    env.close()

# Print out your total sum of rewards here
assert len(rewards_list) > 0, f"Assertion Error: len(rewards_list) = {len(rewards_list)}"
print(f"[Random Policy] The maximum of the total reward: {np.max(rewards_list)}")
print(f"[Random Policy] The minimum of the total reward: {np.min(rewards_list)}")
print(f"[Random Policy] The average of the total reward: {np.mean(rewards_list)}")
print(f"[Random Policy] The standard deviation of the total reward: {np.std(rewards_list)}")

[Random Policy] The maximum of the total reward: 41.0
[Random Policy] The minimum of the total reward: 10.0
[Random Policy] The average of the total reward: 19.7
[Random Policy] The standard deviation of the total reward: 9.445104552094698


In [5]:
# 1.1 [10pts] (Part 3/3)

max_episode = 10  # run several times
# max_rewards = 0
rewards_list = []

for ep_idx in range(max_episode):
    env = gym.make("CartPole-v1", render_mode="human")
    # runs a single episode and render it. Try running this before editing anything
    obs, info = env.reset(seed=RANDOM_SEED + ep_idx)  # get first obs/state; set random seed for the env
    rewards_sum = 0  # The sum of rewards in this episode
    while True:
        # Replace this `action` with something that depends on `obs` 
        # action = env.action_space.sample()  # random action

        # A straightforward idea: if pole angle <= 0, the pole will lean towards left,
        # so we push the cart to the left. Otherwise, we push the cart to the right.
        action = 0 if obs[2] < 0 else 1

        obs, reward, terminated, truncated, info = env.step(action)
        rewards_sum += reward  # reward cumulation

        env.render()
        time.sleep(0.1)  # so it doesn't render too quickly

        if terminated | truncated:
            break

    # Done simulation of this episode
    rewards_list.append(rewards_sum)
    env.close()

# Print out your total sum of rewards here
assert len(rewards_list) > 0, f"Assertion Error: len(rewards_list) = {len(rewards_list)}"
print(f"[Random Policy] The maximum of the total reward: {np.max(rewards_list)}")
print(f"[Random Policy] The minimum of the total reward: {np.min(rewards_list)}")
print(f"[Random Policy] The average of the total reward: {np.mean(rewards_list)}")
print(f"[Random Policy] The standard deviation of the total reward: {np.std(rewards_list)}")

[Random Policy] The maximum of the total reward: 56.0
[Random Policy] The minimum of the total reward: 32.0
[Random Policy] The average of the total reward: 42.9
[Random Policy] The standard deviation of the total reward: 8.092589202474075


To answer the questions below, look at the full [source code here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py) if you haven't already.

**1.2. [2pts] Briefly describe your policy.  What observation information does it use?  What score did you achieve (rough maximum and average)?  And how does it compare to the random policy?**

According to the [document](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py), there are two actions in the Action Space, i.e., 0 (Push cart to the left) and 1 (Push cart to the right), and there are four obervations in the Observation Space, i.e., 0 (Cart Position $\in$ [-4.8, 4.8]), 1 (Cart Velocity $\in$ (-Inf, Inf)), 2 (Pole Angle $\in$ [-0.418, 0.418], rad (-24°) to rad (24°)), and 3 (Pole Angular Velocity $\in$ (-Inf, Inf)).

**Q 1.2.1.: What observation information does it use?**

**Answer**: Here, `obs[2]` (**Pole Angle**) is used to determine which action to take.

The idea of taking actions is to push the cart to the left if pole angle <= 0, which means the pole will lean towards left. Otherwise, we push the cart to the right.

**Q 1.2.2.: What score did you achieve (rough maximum and average)?**

**Answer**: After running the experiments for 10 times, the maximum of the total reward is 56.0, and the average of the total reward is 42.9.

**Q 1.2.3.: And how does it compare to the random policy?**

**Answer**: Compared to the random policy (Max: 41.0; Min: 10.0; Avg: 19.7; Std: 9.45), taking action based on the pole angle (Max: 56.0; Min: 32.0; Avg: 42.9; Std: 8.09) **performs much better** and **more stable** (lower std).

---

##  [12 pts] Discretize the env

Next, we need to discretize CartPole's continuous state space to work for tabular Q-learning.  While this is in part  a contrived usage of tabular methods, given the existence of other approaches that are designed to cope with continuous state-spaces, it is also interesting to consider whether tabular methods can be adapted more directly via discretization of the state into bins. Furthermore, tabular methods are simple, interpretabile, and can be proved to converge, and thus they still remain relevant.

Your task is to discretize the state/observation space so that it is compatible with tabular Q-learning.  To do this:
- **implement `obs_normalizer` to pass its test**
- **implement `get_bins` to pass its test**
- **then answer question 2.3**

- [map] : https://arxiv.org/abs/1504.04909
- [qd] : https://quality-diversity.github.io/

In [2]:
env = gym.make("CartPole-v1")
# obs, info = env.reset(seed=RANDOM_SEED)  # get first obs/state; set random seed for the env
# rewards_sum = 0  # The sum of rewards in this episode

In [3]:
# 2.1 [5 pts for passing test_normed]

def obs_normalizer(obs):
    """Normalize the observations between 0 and 1

    If the observation has extremely large bounds, then clip to a reasonable range before normalizing; 
    (-2,2) should work. (It is OK if the solution is specific to CartPole)

    Args:
        obs (np.ndarray): shape (4,) containing an observation from CartPole using the bound of the env
    Returns:
        normed (np.ndarray): shape (4,) where all elements are roughly uniformly mapped to the range [0, 1]
    """
    # HINT: check out env.observation_space.high, env.observation_space.low

    hi, lo = env.observation_space.high, env.observation_space.low
    clipper = lambda x: np.clip(x, -2, 2)  # clip to a reasonable range
    obs[[1, 3]] = clipper(obs[[1, 3]])
    hi[[1, 3]] = clipper(hi[[1, 3]])
    lo[[1, 3]] = clipper(lo[[1, 3]])

    for i in range(len(obs)):
        obs[i] = (obs[i] - lo[i]) / (hi[i] - lo[i])  # normalization

    return obs


In [4]:
### TEST 2.1

env = gym.make("CartPole-v1")

def test_normed():
    obs, info = env.reset()
    while True:
        obs, _, terminated, truncated, _ =  env.step(env.action_space.sample())
        normed = obs_normalizer(obs) 
        assert np.all(normed >= 0.0) and np.all(normed <= 1.0), '{} are outside of (0,1)'.format(normed)
        if terminated | truncated: break
    print('Passed!')

test_normed()
env.close()

Passed!


In [5]:
# 2.2 [5 pts for passing test_binned]

def get_bins(normed, num_bins):
    """Map normalized observations (0, 1) to bin index values (0, num_bins-1)

    Args:
        normed (np.ndarray): shape (4,) output from obs_normalizer
        num_bins (int): how many bins to use
    Returns:
        binned (np.ndarray of type np.int32): shape (4,) where all elements are values in range [0, num_bins - 1]
    """

    normed *= num_bins - 1
    binned = np.rint(normed).astype(np.int32)

    return binned


In [6]:
### TEST 2.2

env = gym.make("CartPole-v1")
obs, info = env.reset()

def test_binned(num_bins):
    normed = np.array([0.0, 0.2, 0.8, 1.0])
    binned = get_bins(normed, num_bins)
    assert np.all(binned >= 0) and np.all(binned < num_bins), '{} supposed to be between (0, {})'.format(binned, num_bins - 1)
    assert binned.dtype == np.int32, "You should also make sure to cast your answer to int using arr.astype(np.int32)" 

test_binned(5)
test_binned(10)
test_binned(50)
print('Passed!')
env.close()

Passed!


**2.3. [2 pts] If your state has 4 values and each is binned into N possible bins, how many bins are needed to represent all unique possible states?**



**Answer**: $N^4$ bins are needed to represent all unique possible states, because each of the 4 continuous state values can be discretized into N possible bins.

---

## [20 pts] Solve the env 

Using the pseudocode below and the functions you implemented above, implement tabular Q-learning and use it to solve CartPole.

We provide setup code to initialize the Q-table and give examples of interfacing with it. Write the inner and outer loops to train your algorithm.  These training loops will be similar to those deep RL approaches, so get used to writing them!

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![Sutton RL](https://i.imgur.com/mdcWVRL.png)

in summary:
- **implement Q-learning using this pseudocode and the helper code**
- **answer the questions below**
- **run the suggested experiments and otherwise experiment with whatever interests you**

In [7]:
# setup (see last few lines for how to use the Q-table)
env = gym.make("CartPole-v1")

# hyper parameters. feel free to change these as desired and experiment with different values
num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000
# epsilon greedy
eps = 0.05  #usage: action = optimal if np.random.rand() > eps else random

obs, info = env.reset(seed=RANDOM_SEED)

# Q-table initialized to zeros.  first 4 dims are state, last dim is for action (0,1) for left,right.
Q = np.zeros([num_bins] * len(obs) + [env.action_space.n])

# helper function to convert observation into a binned state so we can index into our Q-table
obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

s = obs2bin(obs)

print(f"Shape of Q Table: {Q.shape}") # you can imagine why tabular learning does not scale very well
print(f"Original obs {obs} --> binned {s}")
print(f"Value of Q Table at that obs/state value: {Q[s]}")

env.close()

Shape of Q Table: (10, 10, 10, 10, 2)
Original obs [4.5256834 4.486248  4.8852406 4.544408 ] --> binned (5, 4, 5, 5)
Value of Q Table at that obs/state value: [0. 0.]


In [8]:
# 3.1 [20 pts]

# Task: implement Q learning, following the pseudo-code above. 
#     - you can follow it almost exactly, but translating things for the gym api and our code used above
#     - make sure to use e-greedy, where e = random about 0.05 percent of the time
#     - make sure to do the S <-- S' step because it can be easy to forget
#     - every log_n steps, you should render your environment and
#       print out the average total episode rewards of the past log_n runs to monitor how your agent trains
#      (your implementation should be able to break at least +150 average reward value, and you can use that 
#       as a breaking condition.  It make take several minutes to run depending on your computer.)

# env = gym.make("CartPole-v1")
# # hyper parameters. feel free to change these as desired and experiment with different values
# num_bins = 10
# alpha = 0.1
# gamma = 0.99
# log_n = 1000
# # epsilon greedy
# eps = 0.05  # usage: action = optimal if np.random.rand() > eps else random
# obs, info = env.reset(seed=RANDOM_SEED)
# # Q-table initialized to zeros.  first 4 dims are state, last dim is for action (0,1) for left,right.
# Q = np.zeros([num_bins] * len(obs) + [env.action_space.n])

episode = 0
rewards_list = []

while True:
    env = gym.make("CartPole-v1")  # do NOT render when training
    obs, info = env.reset(seed=RANDOM_SEED + episode)
    rewards_sum = 0  # The sum of rewards in this episode
    s = obs2bin(obs)  # convert the observation into a binned state (init state)

    while True:
        if np.random.rand() < eps:  # e-greedy
            action = env.action_space.sample()  # random action
        else:
            action = np.random.choice([0, 1]) if Q[s][0] == Q[s][1] else np.argmax(Q[s])  # Q-learning policy

        obs, reward, terminated, truncated, info = env.step(action)
        rewards_sum += reward  # reward cumulation

        s_new = obs2bin(obs)  # get the new state
        Q[s][action] = Q[s][action] + alpha * (reward + gamma * np.max(Q[s_new]) - Q[s][action])  # update Q table
        s = s_new  # update the state

        if terminated | truncated:
            break

    # Done simulation of this episode
    rewards_list.append(rewards_sum)
    episode += 1
    # env.close()

    # print out the average total episode rewards of the past log_n runs
    if episode % log_n == 0:
        # render the environment without updating the Q-learning policy (Q table)
        env = gym.make("CartPole-v1", render_mode="human")  # do render when evaluating
        obs, info = env.reset(seed=RANDOM_SEED + episode - 1)
        s = obs2bin(obs)  # convert the observation into a binned state (init state)
        while True:
            if np.random.rand() < eps:  # e-greedy
                action = env.action_space.sample()  # random action
            else:
                action = np.random.choice([0, 1]) if Q[s][0] == Q[s][1] else np.argmax(Q[s])  # Q-learning policy

            obs, reward, terminated, truncated, info = env.step(action)
            s_new = obs2bin(obs)  # get the new state
            # Q[s][action] = Q[s][action] + alpha * (reward + gamma * np.max(Q[s_new]) - Q[s][action])  # NOT to update Q table
            s = s_new  # update the state

            env.render()
            time.sleep(0.1)  # so it doesn't render too quickly

            if terminated | truncated:
                break
        # env.close()

        # only consider these log_n steps
        cur_rewards_list = rewards_list[episode - log_n:]
        r_max, r_min = np.max(cur_rewards_list), np.min(cur_rewards_list)
        r_avg, r_std = np.mean(cur_rewards_list), np.std(cur_rewards_list)
        print(f"[Episode: {episode}] Rewards of rencent {log_n} steps: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

        if r_avg >= 150:
            break  # break at least +150 average reward value, else the Q-learning process should continue
        else:
            total_rewards = 0

# Print out the statistics of the rewards
assert len(rewards_list) > 0, f"Assertion Error: len(rewards_list) = {len(rewards_list)}"
r_max, r_min = np.max(rewards_list), np.min(rewards_list)
r_avg, r_std = np.mean(rewards_list), np.std(rewards_list)
print(f"END\n[Episode: {episode}] Overall Rewards Statistics: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

[Episode: 1000] Rewards of rencent 1000 steps: Avg: 77.408; Max: 335.0; Min: 8.0; Std: 65.53388082511213
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 110.641; Max: 455.0; Min: 9.0; Std: 67.089403924912
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 135.318; Max: 500.0; Min: 11.0; Std: 61.01221907126474
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 154.35; Max: 500.0; Min: 11.0; Std: 81.42237714535237
END
[Episode: 4000] Overall Rewards Statistics: Avg: 119.42925; Max: 500.0; Min: 8.0; Std: 74.9378108463111


## [10 pts] Experiments

Given a working algorithm, you will run a few experiments.  Either make a copy of your code above to modify, or make the modifications in a way that they can be commented out or switched between (with boolean flag if statements).

**4.2. [5 pts] $\epsilon$-greedy.**  How sensitive are the results to the value of $\epsilon$?   First, write down your prediction of what would happen if $\epsilon$ is set to various values, including for example [0, 0.05, 0.25, 0.5].

**Answer**: Since larger $\epsilon$ will cause more on random action be taken, I guess $\epsilon = 0.25$ or $\epsilon = 0.5$ should have bad results compared to $\epsilon = 0.05$. As for $\epsilon = 0$, there is no randomly exploration $\to$ the result could be bad because I believe using the $\epsilon$-greedy method is better (especially in the beginning RL stages).

Now run the experiment and observe the impact on the algorithm.  Report the results below.

In [15]:
env = gym.make("CartPole-v1")

num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000

obs, info = env.reset(seed=RANDOM_SEED)
Q = np.zeros([num_bins] * len(obs) + [env.action_space.n])

# epsilon greedy
# eps = 0.0
eps = 0.05
# eps = 0.25
# eps = 0.5

episode = 0
rewards_list = []

while True:
    # env = gym.make("CartPole-v1")  # do NOT render when training
    obs, info = env.reset(seed=RANDOM_SEED + episode)
    rewards_sum = 0  # The sum of rewards in this episode
    s = obs2bin(obs)  # convert the observation into a binned state (init state)

    while True:
        if np.random.rand() < eps:  # e-greedy
            action = env.action_space.sample()  # random action
        else:
            action = np.random.choice([0, 1]) if Q[s][0] == Q[s][1] else np.argmax(Q[s])  # Q-learning policy

        obs, reward, terminated, truncated, info = env.step(action)
        rewards_sum += reward  # reward cumulation

        s_new = obs2bin(obs)  # get the new state
        Q[s][action] = Q[s][action] + alpha * (reward + gamma * np.max(Q[s_new]) - Q[s][action])  # update Q table
        s = s_new  # update the state

        if terminated | truncated:
            break

    # Done simulation of this episode
    rewards_list.append(rewards_sum)
    episode += 1
    # env.close()

    # print out the average total episode rewards of the past log_n runs
    if episode % log_n == 0:
        # only consider these log_n steps
        cur_rewards_list = rewards_list[episode - log_n:]
        r_max, r_min = np.max(cur_rewards_list), np.min(cur_rewards_list)
        r_avg, r_std = np.mean(cur_rewards_list), np.std(cur_rewards_list)
        print(f"[Episode: {episode}] Rewards of rencent {log_n} steps: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

        if r_avg >= 150:
            break  # break at least +150 average reward value, else the Q-learning process should continue
        else:
            total_rewards = 0

# Print out the statistics of the rewards
assert len(rewards_list) > 0, f"Assertion Error: len(rewards_list) = {len(rewards_list)}"
r_max, r_min = np.max(rewards_list), np.min(rewards_list)
r_avg, r_std = np.mean(rewards_list), np.std(rewards_list)
print(f"END\n[Episode: {episode}] Overall Rewards Statistics: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

[Episode: 1000] Rewards of rencent 1000 steps: Avg: 41.802; Max: 229.0; Min: 8.0; Std: 32.18572348107154
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 118.953; Max: 500.0; Min: 8.0; Std: 70.85190746197311
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 206.984; Max: 500.0; Min: 13.0; Std: 168.53187159703648
END
[Episode: 3000] Overall Rewards Statistics: Avg: 122.57966666666667; Max: 500.0; Min: 8.0; Std: 126.6506756919292


**Answer**: The result analysis and experimental outputs are as follows:

- $\epsilon = 0$: The learning process is stuck. The reason could be the lack of exploration.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 52.911; Max: 257.0; Min: 13.0; Std: 32.45712678288083
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 54.805; Max: 309.0; Min: 13.0; Std: 33.98250983962191
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 53.636; Max: 327.0; Min: 13.0; Std: 33.96947900689677
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 53.744; Max: 231.0; Min: 12.0; Std: 31.49305421835107
[Episode: 5000] Rewards of rencent 1000 steps: Avg: 53.193; Max: 204.0; Min: 13.0; Std: 31.305139370397313
[Episode: 6000] Rewards of rencent 1000 steps: Avg: 52.464; Max: 284.0; Min: 13.0; Std: 32.77911993937604
[Episode: 7000] Rewards of rencent 1000 steps: Avg: 52.372; Max: 273.0; Min: 14.0; Std: 31.092050688238626
[Episode: 8000] Rewards of rencent 1000 steps: Avg: 51.958; Max: 291.0; Min: 13.0; Std: 31.875103701792092
[Episode: 9000] Rewards of rencent 1000 steps: Avg: 52.966; Max: 315.0; Min: 11.0; Std: 31.771793213477896
[Episode: 10000] Rewards of rencent 1000 steps: Avg: 54.382; Max: 306.0; Min: 13.0; Std: 34.24532195789667
[Episode: 11000] Rewards of rencent 1000 steps: Avg: 53.785; Max: 335.0; Min: 13.0; Std: 32.86601245968242
[Episode: 12000] Rewards of rencent 1000 steps: Avg: 53.715; Max: 294.0; Min: 11.0; Std: 32.01302508354998
[Episode: 13000] Rewards of rencent 1000 steps: Avg: 51.671; Max: 287.0; Min: 13.0; Std: 31.492106296657898
[Episode: 14000] Rewards of rencent 1000 steps: Avg: 53.992; Max: 344.0; Min: 13.0; Std: 35.02967222227465
[Episode: 15000] Rewards of rencent 1000 steps: Avg: 55.579; Max: 289.0; Min: 13.0; Std: 35.21354510696133
[Episode: 16000] Rewards of rencent 1000 steps: Avg: 54.759; Max: 251.0; Min: 13.0; Std: 31.34700813474868
[Episode: 17000] Rewards of rencent 1000 steps: Avg: 53.812; Max: 312.0; Min: 12.0; Std: 34.26847904415952
[Episode: 18000] Rewards of rencent 1000 steps: Avg: 54.125; Max: 266.0; Min: 13.0; Std: 32.12269252413316
[Episode: 19000] Rewards of rencent 1000 steps: Avg: 53.444; Max: 269.0; Min: 13.0; Std: 32.17410238064148
[Episode: 20000] Rewards of rencent 1000 steps: Avg: 53.209; Max: 276.0; Min: 13.0; Std: 31.14294974789639
...
```

- $\epsilon = 0.05$: It works well.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 41.802; Max: 229.0; Min: 8.0; Std: 32.18572348107154
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 118.953; Max: 500.0; Min: 8.0; Std: 70.85190746197311
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 206.984; Max: 500.0; Min: 13.0; Std: 168.53187159703648
END
```

- $\epsilon = 0.25$: Surprisingly, it also works well.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 60.309; Max: 342.0; Min: 11.0; Std: 48.91293815546149
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 141.06; Max: 500.0; Min: 13.0; Std: 78.36450982428207
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 149.569; Max: 500.0; Min: 11.0; Std: 77.80620308818571
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 178.806; Max: 500.0; Min: 14.0; Std: 88.41591691545138
END
```

- $\epsilon = 0.5$: The learning process is stuck in the 90-100 average rewards. The reason could be the policy relies too much (50%) on random actions.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 34.801; Max: 200.0; Min: 9.0; Std: 26.97886949076999
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 71.401; Max: 249.0; Min: 10.0; Std: 44.917437582747304
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 84.584; Max: 323.0; Min: 10.0; Std: 51.644621636720316
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 92.898; Max: 443.0; Min: 10.0; Std: 57.81867860821449
[Episode: 5000] Rewards of rencent 1000 steps: Avg: 95.868; Max: 342.0; Min: 10.0; Std: 59.2893462267885
[Episode: 6000] Rewards of rencent 1000 steps: Avg: 96.933; Max: 417.0; Min: 12.0; Std: 62.735480479550006
[Episode: 7000] Rewards of rencent 1000 steps: Avg: 97.453; Max: 500.0; Min: 9.0; Std: 62.918215097060724
[Episode: 8000] Rewards of rencent 1000 steps: Avg: 100.913; Max: 491.0; Min: 10.0; Std: 65.5409141758032
[Episode: 9000] Rewards of rencent 1000 steps: Avg: 96.946; Max: 453.0; Min: 9.0; Std: 64.17157535856511
[Episode: 10000] Rewards of rencent 1000 steps: Avg: 96.936; Max: 500.0; Min: 10.0; Std: 67.60065017438812
[Episode: 11000] Rewards of rencent 1000 steps: Avg: 102.022; Max: 500.0; Min: 10.0; Std: 73.41876814548172
[Episode: 12000] Rewards of rencent 1000 steps: Avg: 97.709; Max: 496.0; Min: 10.0; Std: 68.1936677338886
[Episode: 13000] Rewards of rencent 1000 steps: Avg: 98.353; Max: 380.0; Min: 10.0; Std: 66.62202632012928
[Episode: 14000] Rewards of rencent 1000 steps: Avg: 98.339; Max: 500.0; Min: 10.0; Std: 69.37309333596131
[Episode: 15000] Rewards of rencent 1000 steps: Avg: 97.417; Max: 403.0; Min: 10.0; Std: 67.63736475499323
[Episode: 16000] Rewards of rencent 1000 steps: Avg: 91.735; Max: 395.0; Min: 9.0; Std: 63.51775165258921
[Episode: 17000] Rewards of rencent 1000 steps: Avg: 93.245; Max: 409.0; Min: 9.0; Std: 66.60620823166562
[Episode: 18000] Rewards of rencent 1000 steps: Avg: 90.028; Max: 428.0; Min: 10.0; Std: 65.2411773039083
[Episode: 19000] Rewards of rencent 1000 steps: Avg: 92.095; Max: 337.0; Min: 9.0; Std: 62.27835880143278
[Episode: 20000] Rewards of rencent 1000 steps: Avg: 91.84; Max: 359.0; Min: 9.0; Std: 60.01928356786675
```


**4.3. [5 pts] Design your own experiment.** Design a modification that you think would either increase or reduce performance.  A simple example (which you can use) is initializing the Q-table differently, and thinking about how this might alter performance. Write down your idea, what you think might happen, and why.

**Answer**: $\alpha$ stands for the learning rate. I think the current learning rate of $0.1$ is a moderate choice. $\alpha=0.05$ and $\alpha=0.2$ could also be good choices. However, $\alpha=0.01$ might reduce the performance because the learning is too slow. $\alpha=0.5$ will speed up the learning process at first, but such large learning might make the process hard to converge.

Run the experiment and report the results.

In [30]:
env = gym.make("CartPole-v1")

num_bins = 10
gamma = 0.99
log_n = 1000
eps = 0.05  # epsilon greedy

obs, info = env.reset(seed=RANDOM_SEED)
Q = np.zeros([num_bins] * len(obs) + [env.action_space.n])

# learning rate
# alpha = 0.01
# alpha = 0.05
# alpha = 0.1
alpha = 0.2
# alpha = 0.5

episode = 0
rewards_list = []

while True:
    # env = gym.make("CartPole-v1")  # do NOT render when training
    obs, info = env.reset(seed=RANDOM_SEED + episode)
    rewards_sum = 0  # The sum of rewards in this episode
    s = obs2bin(obs)  # convert the observation into a binned state (init state)

    while True:
        if np.random.rand() < eps:  # e-greedy
            action = env.action_space.sample()  # random action
        else:
            action = np.random.choice([0, 1]) if Q[s][0] == Q[s][1] else np.argmax(Q[s])  # Q-learning policy

        obs, reward, terminated, truncated, info = env.step(action)
        rewards_sum += reward  # reward cumulation

        s_new = obs2bin(obs)  # get the new state
        Q[s][action] = Q[s][action] + alpha * (reward + gamma * np.max(Q[s_new]) - Q[s][action])  # update Q table
        s = s_new  # update the state

        if terminated | truncated:
            break

    # Done simulation of this episode
    rewards_list.append(rewards_sum)
    episode += 1
    # env.close()

    # print out the average total episode rewards of the past log_n runs
    if episode % log_n == 0:
        # only consider these log_n steps
        cur_rewards_list = rewards_list[episode - log_n:]
        r_max, r_min = np.max(cur_rewards_list), np.min(cur_rewards_list)
        r_avg, r_std = np.mean(cur_rewards_list), np.std(cur_rewards_list)
        print(f"[Episode: {episode}] Rewards of rencent {log_n} steps: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

        if r_avg >= 150:
            break  # break at least +150 average reward value, else the Q-learning process should continue
        else:
            total_rewards = 0

# Print out the statistics of the rewards
assert len(rewards_list) > 0, f"Assertion Error: len(rewards_list) = {len(rewards_list)}"
r_max, r_min = np.max(rewards_list), np.min(rewards_list)
r_avg, r_std = np.mean(rewards_list), np.std(rewards_list)
print(f"END\n[Episode: {episode}] Overall Rewards Statistics: Avg: {r_avg}; Max: {r_max}; Min: {r_min}; Std: {r_std}")

[Episode: 1000] Rewards of rencent 1000 steps: Avg: 128.55; Max: 500.0; Min: 12.0; Std: 72.25274735260936
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 161.046; Max: 500.0; Min: 11.0; Std: 85.87460558279146
END
[Episode: 2000] Overall Rewards Statistics: Avg: 144.798; Max: 500.0; Min: 11.0; Std: 81.00278511261203


**Answer**: The result analysis and experimental outputs are as follows:

- $\epsilon = 0.01$: The learning process is stuck.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 31.591; Max: 154.0; Min: 9.0; Std: 27.55648234082137
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 36.381; Max: 161.0; Min: 9.0; Std: 30.628742040769485
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 32.745; Max: 159.0; Min: 8.0; Std: 29.095325655506933
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 32.961; Max: 185.0; Min: 9.0; Std: 29.46084654248754
[Episode: 5000] Rewards of rencent 1000 steps: Avg: 32.458; Max: 148.0; Min: 8.0; Std: 28.573908308105143
[Episode: 6000] Rewards of rencent 1000 steps: Avg: 33.89; Max: 165.0; Min: 9.0; Std: 29.790466595875937
[Episode: 7000] Rewards of rencent 1000 steps: Avg: 32.041; Max: 167.0; Min: 8.0; Std: 28.67004218692397
[Episode: 8000] Rewards of rencent 1000 steps: Avg: 34.262; Max: 154.0; Min: 9.0; Std: 30.40857372518481
[Episode: 9000] Rewards of rencent 1000 steps: Avg: 34.198; Max: 159.0; Min: 8.0; Std: 30.912340513134875
[Episode: 10000] Rewards of rencent 1000 steps: Avg: 34.322; Max: 157.0; Min: 9.0; Std: 31.08688334330092
[Episode: 11000] Rewards of rencent 1000 steps: Avg: 36.403; Max: 161.0; Min: 9.0; Std: 31.74118131071999
[Episode: 12000] Rewards of rencent 1000 steps: Avg: 38.044; Max: 164.0; Min: 8.0; Std: 30.56599522345052
[Episode: 13000] Rewards of rencent 1000 steps: Avg: 56.494; Max: 163.0; Min: 9.0; Std: 40.13041195901184
[Episode: 14000] Rewards of rencent 1000 steps: Avg: 67.694; Max: 268.0; Min: 9.0; Std: 41.96861165204301
[Episode: 15000] Rewards of rencent 1000 steps: Avg: 79.659; Max: 242.0; Min: 9.0; Std: 39.59369544510843
[Episode: 16000] Rewards of rencent 1000 steps: Avg: 79.388; Max: 290.0; Min: 10.0; Std: 40.58698628871082
[Episode: 17000] Rewards of rencent 1000 steps: Avg: 87.404; Max: 283.0; Min: 11.0; Std: 38.886485878772845
[Episode: 18000] Rewards of rencent 1000 steps: Avg: 90.162; Max: 201.0; Min: 11.0; Std: 35.92500182324282
[Episode: 19000] Rewards of rencent 1000 steps: Avg: 83.582; Max: 191.0; Min: 10.0; Std: 29.866256477837997
[Episode: 20000] Rewards of rencent 1000 steps: Avg: 84.852; Max: 195.0; Min: 10.0; Std: 31.713405619705995
```

- $\epsilon = 0.05$: It works well.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 58.16; Max: 255.0; Min: 10.0; Std: 39.490029121285794
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 70.113; Max: 315.0; Min: 11.0; Std: 46.93847282347392
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 102.607; Max: 325.0; Min: 16.0; Std: 61.91047206248714
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 164.633; Max: 371.0; Min: 18.0; Std: 61.61537398247292
END
```

- $\epsilon = 0.1$: It works better.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 24.23; Max: 150.0; Min: 8.0; Std: 18.3809983406778
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 81.586; Max: 446.0; Min: 8.0; Std: 69.1534424595045
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 161.398; Max: 500.0; Min: 12.0; Std: 111.24750602148346
END
```

- $\epsilon = 0.2$: It works the best.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 128.55; Max: 500.0; Min: 12.0; Std: 72.25274735260936
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 161.046; Max: 500.0; Min: 11.0; Std: 85.87460558279146
END
```

- $\epsilon = 0.5$: It learns fast at first but gets stuck eventually.

```txt
[Episode: 1000] Rewards of rencent 1000 steps: Avg: 84.624; Max: 500.0; Min: 10.0; Std: 63.07727502040652
[Episode: 2000] Rewards of rencent 1000 steps: Avg: 95.759; Max: 500.0; Min: 9.0; Std: 64.26968895988217
[Episode: 3000] Rewards of rencent 1000 steps: Avg: 79.304; Max: 500.0; Min: 11.0; Std: 46.70870993722691
[Episode: 4000] Rewards of rencent 1000 steps: Avg: 87.207; Max: 500.0; Min: 10.0; Std: 53.46432596601213
[Episode: 5000] Rewards of rencent 1000 steps: Avg: 86.681; Max: 500.0; Min: 10.0; Std: 68.7742774516752
[Episode: 6000] Rewards of rencent 1000 steps: Avg: 83.72; Max: 500.0; Min: 13.0; Std: 58.50522711689956
[Episode: 7000] Rewards of rencent 1000 steps: Avg: 85.714; Max: 500.0; Min: 10.0; Std: 62.77335584465753
[Episode: 8000] Rewards of rencent 1000 steps: Avg: 92.396; Max: 500.0; Min: 10.0; Std: 70.86133772375454
[Episode: 9000] Rewards of rencent 1000 steps: Avg: 92.919; Max: 500.0; Min: 10.0; Std: 53.231517346399215
[Episode: 10000] Rewards of rencent 1000 steps: Avg: 89.834; Max: 369.0; Min: 14.0; Std: 51.48633259419435
[Episode: 11000] Rewards of rencent 1000 steps: Avg: 91.212; Max: 500.0; Min: 12.0; Std: 62.7905809496934
[Episode: 12000] Rewards of rencent 1000 steps: Avg: 103.315; Max: 500.0; Min: 12.0; Std: 74.67600534977751
[Episode: 13000] Rewards of rencent 1000 steps: Avg: 89.908; Max: 500.0; Min: 9.0; Std: 65.09316043948088
[Episode: 14000] Rewards of rencent 1000 steps: Avg: 101.242; Max: 500.0; Min: 12.0; Std: 82.95273012987577
[Episode: 15000] Rewards of rencent 1000 steps: Avg: 111.552; Max: 500.0; Min: 11.0; Std: 82.85846544560188
[Episode: 16000] Rewards of rencent 1000 steps: Avg: 126.713; Max: 500.0; Min: 11.0; Std: 93.7427150822932
[Episode: 17000] Rewards of rencent 1000 steps: Avg: 115.065; Max: 500.0; Min: 12.0; Std: 94.69150318270377
[Episode: 18000] Rewards of rencent 1000 steps: Avg: 94.914; Max: 500.0; Min: 10.0; Std: 70.53345733763517
[Episode: 19000] Rewards of rencent 1000 steps: Avg: 88.733; Max: 500.0; Min: 11.0; Std: 69.09022876644714
[Episode: 20000] Rewards of rencent 1000 steps: Avg: 97.091; Max: 500.0; Min: 11.0; Std: 78.4708016971918
```

---

## A. Extensions (fully optional, will not be graded, if you have time after Part 2)

- plots your learning curve, using e.g., matplotlib 
- visualize the Q-table to see which values are being updated and not
- design a better binning strategy that uses fewer bins for a better-performing policy
- extend this approach to work on different environments (e.g., LunarLander-v2)
- extend this approach to work on environments with continuous actions, by using a fixed set of discrete samples of the action space.  e.g., for Pendulum-v0
- implement a simple deep learning version of this.  we will see next part that DQN uses some tricks to make the neural network training more stable.  Experiment directly with simply replacing the Q-table with a Q-Network and train the Q-Network using gradient descent with `loss = (targets - Q(s,a))**2`, where `targets = stop_grad(R + gamma * maxa(Q(s,a))`).

# Part 2 [60 pts] Behavioral Cloning and Deep Q Learning

---
The second part of assignment will help you transition from tabular approaches to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this part we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## 1. [36 pts] Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### [10 pts] 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

**[6 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER TO INSERT YOUR __getitem__ method here

# def __getitem__(self, index):
#     item = self.data[index]
#     # YOUR CODE HERE
#     raise NotImplementedError()

"""
def __getitem__(self, index):
    item = self.data[index]
    return {
        "state": torch.tensor(item[0], dtype=torch.float),
        "action": torch.tensor(item[1], dtype=torch.long)
    }
"""

**[2 pt]** How big is the dataset provided?

**Answer**: There are 99660 items in the dataset.

**[2 pts]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover? What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?

**Answer**: The dimensionality of state $s$ is 4. The range of each dimension of state $s$ is as follows:

- upper bound: [2.3994859590448474, 1.8469797499257607, 0.14641718032715906, 0.47143313845955753]
- lower bound: [-0.7226705660038144, -0.4330368853821298, -0.050071982930833064, -0.38122098085691347]

The dimensionality of action $a$ is 1 (`int64` scalar). The range of each dimension of action $a$ is $\in \{0, 1\}$.

The code is as follows:

```python
dataset = Dataset(data_path="./{}_dataset.pkl".format("CartPole-v1"))

# The dimensionality of state $s$ is 4
print(len(dataset.data[0][0]))  # 4

# The range of each dimension of state $s$
# upper bound: [2.3994859590448474, 1.8469797499257607, 0.14641718032715906, 0.47143313845955753]
# lower bound: [-0.7226705660038144, -0.4330368853821298, -0.050071982930833064, -0.38122098085691347]
print(
    np.max([item[0][0] for item in dataset.data]), np.min([item[0][0] for item in dataset.data]),
    np.max([item[0][1] for item in dataset.data]), np.min([item[0][1] for item in dataset.data]),
    np.max([item[0][2] for item in dataset.data]), np.min([item[0][2] for item in dataset.data]),
    np.max([item[0][3] for item in dataset.data]), np.min([item[0][3] for item in dataset.data])
)  # 2.3994859590448474, -0.7226705660038144, 1.8469797499257607, -0.4330368853821298, 0.14641718032715906, -0.050071982930833064, 0.47143313845955753, -0.38122098085691347

# The dimensionality of action $a$ is 1
print(type(dataset.data[0][1]))  # <class 'numpy.int64'>

# The range of each dimension of action $a$
print(
    np.max([item[1] for item in dataset.data]), np.min([item[1] for item in dataset.data]),
    np.unique([item[1] for item in dataset.data])
)  # 1, 0, [0, 1]
```

### [5 pts] 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

Considering the full state and action spaces, do you think the provided expert dataset has good coverage? Why or why not? How might this impact the performance of our cloned policy?

**Q 1.2.1.: Do you think the provided expert dataset has good coverage? Why or why not?**

**Answer**: The provided expert dataset has good coverage over the action space because both two actions (pushing left or right) are included. However, regarding the state space, the expert dataset covers only a subset of the real state space.

Specifically, in provided expert dataset, the range of `obs[0]` (Cart Position) is [-0.7226705660038144, 2.3994859590448474] and the range of `obs[2]` (Pole Angle) is [-0.050071982930833064, 0.14641718032715906].

According to the [document](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py), the range of `obs[0]` (Cart Position) is [-4.8, 4.8] and the range of `obs[2]` (Pole Angle) is [-0.418, 0.418].

**Q 1.2.2.: How might this impact the performance of our cloned policy?**

Therefore, the Behavioral Cloning (BC) policy will perform well when the testing data is similar to the expert dataset (OR within this limited value range). However, the BC policy might perform badly otherwise. We could attribute it to the limited range of training data, overfitting to the provided expert dataset, or lack of generalization to real (unseen) situations.

### [14 pts] 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- **[10 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER TO INSERT YOUR MyModel class here

# class MyModel(nn.Module):
#     def __init__(self, state_size, action_size):
#         super(MyModel, self).__init__()
#         # YOUR CODE HERE FOR INITIALIZING THE MODEL

#     def forward(self, x):
#         # YOUR CODE HERE FOR THE FORWARD PASS
#         raise NotImplementedError()

#     def select_action(self, state):
#         self.eval()
#         x = self.forward(state)
#         self.train()
#         return x.max(1)[1].view(1, 1).to(torch.long)

"""
class MyModel(nn.Module):

    def __init__(self, state_size, action_size):
        super().__init__()

        self.fc1 = nn.Linear(state_size, 32)  # input layer
        self.fc2 = nn.Linear(32, 32)  # hidden layer 1
        self.fc3 = nn.Linear(32, 32)  # hidden layer 2
        self.out = nn.Linear(32, action_size)  # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))  # input layer
        x = F.relu(self.fc2(x)) + x  # residual connection
        x = F.relu(self.fc3(x)) + x  # residual connection
        x = self.out(x)  # output layer

        return x

    def select_action(self, state):
        self.eval()
        x = self.forward(state)
        self.train()
        return x.max(1)[1].view(1, 1).to(torch.long)
"""

Answer the following questions:

- **[2 pts]** What is the dimension and meaning of the input of the network?

**Answer**: The dimension of the network input is the same as the size of the state space (`state_size = 4`). (Meaning) It represents the value list of 4 different observations in the cart-pole system: Cart Position, Cart Velocity, Pole Angle, and Pole Angular Velocity.

- **[2 pts]** Similarly, describe the output.

**Answer**: The dimension of the network output is the same as the size of the action space (`action_size = 2`). (Meaning) It represents the value list of 2 different actions in the cart-pole system: Pushing cart to the left, and Pushing cart to the right.

The overall meaning of the network is to absorb the state (input) and decide which action (output) to take.

### [7 pts] 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- **[5 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

# model = None
# optimizer = None
# loss_function = None

"""
model = MyModel(state_size=env.observation_space.shape[0], action_size=env.action_space.n).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.CrossEntropyLoss()
"""

You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

In [ ]:
# Code of BC
"""
def train_behavioral_cloning():
    import os
    import time

    timer_start = time.perf_counter()

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_function = torch.nn.CrossEntropyLoss()

    gradient_steps = 0
    score_list = []
    score_best = 0.0

    for epoch in range(1, TOTAL_EPOCHS + 1):
        for iteration, _data in enumerate(dataloader):
            _data = {k: v.to(device) for k, v in _data.items()}

            output = model(_data["state"])

            loss = loss_function(output, _data["action"])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if gradient_steps % PRINT_INTERVAL == 0:
                print("[epoch {:4d}/{}] [iter {:7d}] [loss {:.5f}]"
                      .format(epoch, TOTAL_EPOCHS, gradient_steps, loss.item()))

            gradient_steps += 1

        if epoch % TEST_INTERVAL == 0:
            score = eval_policy(policy=model, env=ENV_NAME)
            score_list.append(score)
            print("[Test on environment] [epoch {}/{}] [score {:.2f}]"
                  .format(epoch, TOTAL_EPOCHS, score))
            if score > score_best:
                score_best = score
                model_name = "model_best-behavioral_cloning-{}.pt".format(ENV_NAME)
                print("Saving the best model (score: {}) at epoch {} as {}".format(score, epoch, model_name))
                torch.save(model.state_dict(), os.path.join("ckpt", model_name))

    score_max, score_min = np.max(score_list), np.min(score_list)
    score_avg, score_std = np.mean(score_list), np.std(score_list)
    print(f"[END] Score Statistics: Avg = {score_avg}; Max = {score_max}; Min = {score_min}; Std = {score_std}")

    model_name = "model_last-behavioral_cloning-{}.pt".format(ENV_NAME)
    print("Saving the last model as {}".format(model_name))
    torch.save(model.state_dict(), os.path.join("ckpt", model_name))

    timer_end = time.perf_counter()
    time_sec, time_min = timer_end - timer_start, (timer_end - timer_start) / 60
    print(f"\nDONE - Running Time: {time_sec:.1f} sec ({time_min:.1f} min)")
"""

**TERMINAL OUTPUT**: The full output log is here: [bc.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/bc.log)

<a>https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/bc.log</a>

Here, I only paste the first part and the last part.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
# https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/bc.log
"""
[epoch    1/100] [iter       0] [loss 0.72922]
[epoch    1/100] [iter     500] [loss 0.14967]
[epoch    1/100] [iter    1000] [loss 0.08229]
[epoch    1/100] [iter    1500] [loss 0.04913]
[epoch    2/100] [iter    2000] [loss 0.01578]
[epoch    2/100] [iter    2500] [loss 0.01259]
[epoch    2/100] [iter    3000] [loss 0.01122]
[Test on environment] [epoch 2/100] [score 254.00]
Saving the best model (score: 254.0) at epoch 2 as model_best-behavioral_cloning-CartPole-v1.pt

...

[epoch   99/100] [iter  153000] [loss 0.01787]
[epoch   99/100] [iter  153500] [loss 0.00085]
[epoch   99/100] [iter  154000] [loss 0.01833]
[epoch  100/100] [iter  154500] [loss 0.00001]
[epoch  100/100] [iter  155000] [loss 0.00355]
[epoch  100/100] [iter  155500] [loss 0.02178]
[Test on environment] [epoch 100/100] [score 260.40]
[END] Score Statistics: Avg = 256.67400000000004; Max = 298.2; Min = 220.4; Std = 17.38766010709894
Saving the last model as model_last-behavioral_cloning-CartPole-v1.pt

DONE - Running Time: 2558.7 sec (42.6 min)
"""

**[2 pts]** Did you manage to learn a good policy? How consistent is the reward you are getting?

**Answer**: Yes. As the statistics shows, the average is 256.674, the maximum is 298.2, the minimum is 220.4, and the standard deviation is 17.388. The policy learns fast at first, and then the performance fluctuates. Overall, the Behavioral Cloning policy is good, but not as good as Deep Q Learning, as shown in the following sections.

**Run Evaluation:** Behavioral Cloning
```bash
python3 eval_policy.py --model-path "ckpt/model_best-behavioral_cloning-CartPole-v1.pt" --env "CartPole-v1"
```
```txt
[Episode    0/10] [reward 190.0]
[Episode    1/10] [reward 232.0]
[Episode    2/10] [reward 247.0]
[Episode    3/10] [reward 254.0]
[Episode    4/10] [reward 313.0]
[Episode    5/10] [reward 212.0]
[Episode    6/10] [reward 280.0]
[Episode    7/10] [reward 197.0]
[Episode    8/10] [reward 320.0]
[Episode    9/10] [reward 202.0]
```

## 2. [24 pts] Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from [Atari DQN paper](https://arxiv.org/abs/1312.5602)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 [2 pts] Think about your model...


In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

**Answer**: The output of the Behavioral Cloning policy represents the **action decision** based on the input state, while that of DQN represents the **expected rewards (Q value)**. The Q value is calculated by considering the reward gain of each possible action in the current state.

### 2.1 [10 pts] Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR optimize_model function here:

# def optimize_model(state, action, next_state, reward, done):
#     # Given a tuple (s_t, a_t, s_{t+1}, r_t, done_t), update your model weights

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

"""
def optimize_model(state, action, next_state, reward, done):
    # Given a tuple (s_t, a_t, s_{t+1}, r_t, done_t), update your model weights

    # Tensor preproc
    state = torch.FloatTensor(state).unsqueeze(0).to(device)
    action = torch.tensor([action], dtype=torch.long).unsqueeze(1).to(device)
    next_state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
    reward = torch.FloatTensor([reward]).unsqueeze(1).to(device)
    done = torch.FloatTensor([done]).unsqueeze(1).to(device)

    # Q-value calculation
    q_value_pred = model(state).gather(1, action)
    q_value_next_max = target(next_state).detach().max(1)[0].unsqueeze(1)
    q_value_target = reward + (GAMMA * q_value_next_max * (1 - done))

    # Loss computing
    criterion = torch.nn.MSELoss()
    loss = criterion(q_value_pred, q_value_target)

    # Model (parameters) optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
"""

### 2.2 [5 pts] $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR choose_action function here:

# def choose_action(state, test_mode=False):
#     # Implement an epsilon-greedy strategy
#     raise NotImplementedError()

"""
def choose_action(state, test_mode=False):
    # Implement an epsilon-greedy strategy

    if test_mode or random.random() > EPS_EXPLORATION:
        with torch.no_grad():
            model.eval()
            state = torch.FloatTensor(state).unsqueeze(0).to(device)
            action = model(state).max(1)[1].view(1, 1)
    else:  # e-greedy: random action
        action = torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

    return action
"""

### 2.3 [2 pts] Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

How many episodes does it take to learn (ie. reach a good reward)?

**Answer**: As shown in [dqn.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn.log), the DQN model reaches a good reward within 750 episodes.
```txt
[TEST Episode 375] [Average Reward 61.3]
[TEST Episode 475] [Average Reward 209.0]
[TEST Episode 700] [Average Reward 317.4]
[TEST Episode 725] [Average Reward 499.2]
```

**TERMINAL OUTPUT**: The full output log is here: [dqn.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn.log)

<a>https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn.log</a>

Here, I only paste the first part and the last part.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
# https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn.log
"""
[Episode    1/4000] [Steps   12] [reward 13.0]
[Episode    2/4000] [Steps   11] [reward 12.0]
[Episode    3/4000] [Steps    9] [reward 10.0]
[Episode    4/4000] [Steps    8] [reward 9.0]
[Episode    5/4000] [Steps    9] [reward 10.0]
[Episode    6/4000] [Steps    8] [reward 9.0]
[Episode    7/4000] [Steps    9] [reward 10.0]
[Episode    8/4000] [Steps   14] [reward 15.0]
[Episode    9/4000] [Steps    9] [reward 10.0]
[Episode   10/4000] [Steps   10] [reward 11.0]
[Episode   11/4000] [Steps   10] [reward 11.0]
[Episode   12/4000] [Steps   11] [reward 12.0]
[Episode   13/4000] [Steps   15] [reward 16.0]
[Episode   14/4000] [Steps    8] [reward 9.0]
[Episode   15/4000] [Steps   11] [reward 12.0]
[Episode   16/4000] [Steps    9] [reward 10.0]
[Episode   17/4000] [Steps    8] [reward 9.0]
[Episode   18/4000] [Steps   10] [reward 11.0]
[Episode   19/4000] [Steps   11] [reward 12.0]
[Episode   20/4000] [Steps   10] [reward 11.0]
[Episode   21/4000] [Steps    7] [reward 8.0]
[Episode   22/4000] [Steps   10] [reward 11.0]
[Episode   23/4000] [Steps    8] [reward 9.0]
[Episode   24/4000] [Steps   10] [reward 11.0]
[Episode   25/4000] [Steps   10] [reward 11.0]
----------
Saving the best model (score: 9.6) at episode 25 as model_best-dqn-CartPole-v1.pt
[TEST Episode 25] [Average Reward 9.6]
----------

...

----------
[TEST Episode 3975] [Average Reward 9.5]
----------
[Episode 3976/4000] [Steps   12] [reward 13.0]
[Episode 3977/4000] [Steps    9] [reward 10.0]
[Episode 3978/4000] [Steps   13] [reward 14.0]
[Episode 3979/4000] [Steps    8] [reward 9.0]
[Episode 3980/4000] [Steps    8] [reward 9.0]
[Episode 3981/4000] [Steps    7] [reward 8.0]
[Episode 3982/4000] [Steps   11] [reward 12.0]
[Episode 3983/4000] [Steps    9] [reward 10.0]
[Episode 3984/4000] [Steps    9] [reward 10.0]
[Episode 3985/4000] [Steps   10] [reward 11.0]
[Episode 3986/4000] [Steps    8] [reward 9.0]
[Episode 3987/4000] [Steps    9] [reward 10.0]
[Episode 3988/4000] [Steps    9] [reward 10.0]
[Episode 3989/4000] [Steps    8] [reward 9.0]
[Episode 3990/4000] [Steps    8] [reward 9.0]
[Episode 3991/4000] [Steps   12] [reward 13.0]
[Episode 3992/4000] [Steps    8] [reward 9.0]
[Episode 3993/4000] [Steps    9] [reward 10.0]
[Episode 3994/4000] [Steps    7] [reward 8.0]
[Episode 3995/4000] [Steps   11] [reward 12.0]
[Episode 3996/4000] [Steps   14] [reward 15.0]
[Episode 3997/4000] [Steps   11] [reward 12.0]
[Episode 3998/4000] [Steps    8] [reward 9.0]
[Episode 3999/4000] [Steps    8] [reward 9.0]
[Episode 4000/4000] [Steps   10] [reward 11.0]
----------
[TEST Episode 4000] [Average Reward 9.4]
----------

DONE
best_score = 499.2; steps_done = 249931
Running Time: 76.3 sec (1.3 min)
"""

**Further analysis**: As shown in [dqn.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn.log), the performance drops and fluctuates dramatically in the later episodes, possibly because of overfitting.
```txt
[TEST Episode 1050] [Average Reward 473.0]
[TEST Episode 1075] [Average Reward 306.2]
[TEST Episode 1100] [Average Reward 13.0]
[TEST Episode 1350] [Average Reward 483.7]
[TEST Episode 2000] [Average Reward 315.5]
[TEST Episode 2200] [Average Reward 9.2]
```

From the 2200 episode, the average reward is always in the range of 9 to 10.
```txt
[TEST Episode 2200] [Average Reward 9.2]
[TEST Episode 2500] [Average Reward 9.4]
[TEST Episode 3000] [Average Reward 9.3]
[TEST Episode 3500] [Average Reward 9.5]
[TEST Episode 4000] [Average Reward 9.4]
```

According to the [DQN paper](https://arxiv.org/abs/1312.5602), the reason could be:
> "It is easy to see how unwanted feedback loops may arise and the parameters could get stuck in a poor local minimum, or
even diverge catastrophically."

**Run Evaluation:** Deep Q Learning (without Replay Memory)
```bash
python3 eval_policy.py --model-path "ckpt/model_best-dqn-CartPole-v1.pt" --env "CartPole-v1"
```
```txt
[Episode    0/10] [reward 500.0]
[Episode    1/10] [reward 481.0]
[Episode    2/10] [reward 491.0]
[Episode    3/10] [reward 487.0]
[Episode    4/10] [reward 500.0]
[Episode    5/10] [reward 495.0]
[Episode    6/10] [reward 473.0]
[Episode    7/10] [reward 500.0]
[Episode    8/10] [reward 500.0]
[Episode    9/10] [reward 500.0]
```

### 2.4 [5 pts] Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

In [ ]:
# Code of DQN with replay buffer
"""
memory = ReplayBuffer()


def optimize_model_batch(states, actions, next_states, rewards, dones):
    # Given a batch of (s_t, a_t, s_{t+1}, r_t, done_t) tuples, update the model weights

    # Tensor preproc
    states = states.float().to(device)
    actions = actions.long().to(device)
    next_states = next_states.float().to(device)
    rewards = rewards.float().unsqueeze(-1).to(device)
    dones = dones.float().unsqueeze(-1).to(device)

    # Q-value calculation
    q_values_pred = model(states).gather(1, actions)
    q_values_next_max = target(next_states).detach().max(1)[0].unsqueeze(1)
    q_values_target = rewards + (GAMMA * q_values_next_max * (1 - dones))

    # Loss computing
    criterion = torch.nn.MSELoss()
    loss = criterion(q_values_pred, q_values_target)

    # Model (parameters) optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


def train_reinforcement_learning(render=False):
    import os
    import time

    timer_start = time.perf_counter()

    steps_done = 0
    best_score = -float("inf")

    for i_episode in range(1, NUM_EPISODES + 1):
        episode_total_reward = 0
        state, _ = env.reset()
        for t in count():
            action = choose_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action.cpu().numpy()[0][0])
            steps_done += 1
            episode_total_reward += reward

            # optimize_model(state, action, next_state, reward, terminated)  # without replay buffer

            memory.push(state, action, next_state, reward, terminated)  # store transition in replay buffer
            if memory.__len__() > BATCH_SIZE:  # sample random minibatch of transitions from replay buffer
                states, actions, next_states, rewards, dones = memory.sample(BATCH_SIZE)
                optimize_model_batch(states, actions, next_states, rewards, dones)
            else:
                optimize_model(state, action, next_state, reward, done=terminated)

            state = next_state

            if render:
                env.render()

            if terminated or truncated and i_episode % PRINT_INTERVAL == 0:
                print("[Episode {:4d}/{}] [Steps {:4d}] [reward {:.1f}]"
                      .format(i_episode, NUM_EPISODES, t, episode_total_reward))
                break

        if i_episode % TARGET_UPDATE == 0:
            target.load_state_dict(model.state_dict())

        if i_episode % TEST_INTERVAL == 0:
            print("-" * 10)
            score = eval_policy(policy=model, env=ENV_NAME, render=render)
            if score > best_score:
                best_score = score
                model_name = "model_best-dqn_replay-{}.pt".format(ENV_NAME)
                print("Saving the best model (score: {}) at episode {} as {}".format(score, i_episode, model_name))
                torch.save(model.state_dict(), os.path.join("ckpt", model_name))
            print("[TEST Episode {}] [Average Reward {}]".format(i_episode, score))
            print("-" * 10)

    timer_end = time.perf_counter()
    time_sec, time_min = timer_end - timer_start, (timer_end - timer_start) / 60
    print(f"\nDONE\nbest_score = {best_score}; steps_done = {steps_done}\n"
          f"Running Time: {time_sec:.1f} sec ({time_min:.1f} min)")
"""

**TERMINAL OUTPUT**: The full output log is here: [dqn_replay.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn_replay.log)

<a>https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn_replay.log</a>

Here, I only paste the first part and the last part.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
# https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn_replay.log
"""
[Episode    1/4000] [Steps   11] [reward 12.0]
[Episode    2/4000] [Steps   11] [reward 12.0]
[Episode    3/4000] [Steps    8] [reward 9.0]
[Episode    4/4000] [Steps   13] [reward 14.0]
[Episode    5/4000] [Steps    9] [reward 10.0]
[Episode    6/4000] [Steps    9] [reward 10.0]
[Episode    7/4000] [Steps    9] [reward 10.0]
[Episode    8/4000] [Steps   10] [reward 11.0]
[Episode    9/4000] [Steps   10] [reward 11.0]
[Episode   10/4000] [Steps    9] [reward 10.0]
[Episode   11/4000] [Steps    8] [reward 9.0]
[Episode   12/4000] [Steps   11] [reward 12.0]
[Episode   13/4000] [Steps   10] [reward 11.0]
[Episode   14/4000] [Steps   10] [reward 11.0]
[Episode   15/4000] [Steps    9] [reward 10.0]
[Episode   16/4000] [Steps    8] [reward 9.0]
[Episode   17/4000] [Steps   11] [reward 12.0]
[Episode   18/4000] [Steps    9] [reward 10.0]
[Episode   19/4000] [Steps   10] [reward 11.0]
[Episode   20/4000] [Steps   10] [reward 11.0]
[Episode   21/4000] [Steps    9] [reward 10.0]
[Episode   22/4000] [Steps   10] [reward 11.0]
[Episode   23/4000] [Steps    7] [reward 8.0]
[Episode   24/4000] [Steps    8] [reward 9.0]
[Episode   25/4000] [Steps    9] [reward 10.0]
----------
Saving the best model (score: 9.5) at episode 25 as model_best-dqn_reply-CartPole-v1.pt
[TEST Episode 25] [Average Reward 9.5]
----------

...

----------
[TEST Episode 3975] [Average Reward 500.0]
----------
[Episode 3976/4000] [Steps  115] [reward 116.0]
[Episode 3977/4000] [Steps  499] [reward 500.0]
[Episode 3978/4000] [Steps  137] [reward 138.0]
[Episode 3979/4000] [Steps  241] [reward 242.0]
[Episode 3980/4000] [Steps  122] [reward 123.0]
[Episode 3981/4000] [Steps   19] [reward 20.0]
[Episode 3982/4000] [Steps   79] [reward 80.0]
[Episode 3983/4000] [Steps  304] [reward 305.0]
[Episode 3984/4000] [Steps  496] [reward 497.0]
[Episode 3985/4000] [Steps   33] [reward 34.0]
[Episode 3986/4000] [Steps  119] [reward 120.0]
[Episode 3987/4000] [Steps  115] [reward 116.0]
[Episode 3988/4000] [Steps   19] [reward 20.0]
[Episode 3989/4000] [Steps   16] [reward 17.0]
[Episode 3990/4000] [Steps   57] [reward 58.0]
[Episode 3991/4000] [Steps  234] [reward 235.0]
[Episode 3992/4000] [Steps   84] [reward 85.0]
[Episode 3993/4000] [Steps   21] [reward 22.0]
[Episode 3994/4000] [Steps   83] [reward 84.0]
[Episode 3995/4000] [Steps  311] [reward 312.0]
[Episode 3996/4000] [Steps  114] [reward 115.0]
[Episode 3997/4000] [Steps   80] [reward 81.0]
[Episode 3998/4000] [Steps  142] [reward 143.0]
[Episode 3999/4000] [Steps   21] [reward 22.0]
[Episode 4000/4000] [Steps   98] [reward 99.0]
----------
[TEST Episode 4000] [Average Reward 500.0]
----------

DONE
best_score = 500.0; steps_done = 789286
Running Time: 936.8 sec (15.6 min)
"""

How does the replay buffer improve performance?

**Answer**: As shown in [dqn_replay.log](https://github.com/YuweiYin/UBC_CPSC_533V/tree/master/Assignment_2/log/dqn_replay.log), the DQN-replay model reaches a good reward within 450 episodes.
```txt
[TEST Episode 175] [Average Reward 20.4]
[TEST Episode 200] [Average Reward 67.6]
[TEST Episode 250] [Average Reward 215.5]
[TEST Episode 275] [Average Reward 307.6]
[TEST Episode 400] [Average Reward 459.1]
[TEST Episode 425] [Average Reward 500.0]
```

In addition, the performance of the DQN-replay model do NOT drops and fluctuates very dramatically like the vanilla DQN model. According to the [DQN paper](https://arxiv.org/abs/1312.5602), the **reason** could be:
> "By using experience replay the behavior distribution is averaged over many of its previous states, smoothing out learning and avoiding oscillations or divergence in the parameters."

**Run Evaluation:**: Deep Q Learning (with Replay Memory)
```bash
python3 eval_policy.py --model-path "ckpt/model_best-dqn_reply-CartPole-v1.pt" --env "CartPole-v1"
```
```txt
[Episode    0/10] [reward 500.0]
[Episode    1/10] [reward 500.0]
[Episode    2/10] [reward 500.0]
[Episode    3/10] [reward 500.0]
[Episode    4/10] [reward 500.0]
[Episode    5/10] [reward 500.0]
[Episode    6/10] [reward 500.0]
[Episode    7/10] [reward 500.0]
[Episode    8/10] [reward 500.0]
[Episode    9/10] [reward 500.0]
```

## 3. Extra (fully optional)

Ideas to experiment with:

- Is $\epsilon$-greedy strategy the best strategy available? Experiment with other strategies.
- Make use of the model you have trained in the behavioral cloning part and fine-tune it with RL. How does that affect performance?
- You are perhaps bored with `CartPole-v1` by now. Another environment we suggest trying is `LunarLander-v2`. It will be harder to learn but with experimentation, you will find the correct optimizations for success. Piazza is also your friend :)
- What about learning from images? This requires more work because you have to extract the image from the environment. How much more challenging might you expect the learning to be in this case?
- An improvement over DQN is DoubleDQN. Experiment with this to see how much of an impact it makes.



In [ ]:
# YOU CAN USE THIS CODEBLOCK AND ADD ANY BLOCK BELOW AS YOU NEED
# TO SHOW US THE IDEAS AND EXTRA EXPERIMENTS YOU RUN.
# HAVE FUN!

# DONE